In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import numpy as np
import h5py

In [2]:
class DataGenerator(Sequence):
    """Generates data for Keras"""

    def __init__(self, paths, batch_size=16, dim=(64, 64),
                 shuffle=True, k=16):
        """Initialization"""
        self.dim = dim
        self.file_batch_size = 1024
        self.batch_size = batch_size
        self.n_batches = self.file_batch_size // self.batch_size
        self.file_paths = paths
        self.shuffle = shuffle
        self.current_file_loaded = (None, None)
        self.k = k
        self.indexes = np.arange(len(self.file_paths) * int(self.file_batch_size / self.batch_size))
        self.on_epoch_end()

    def load_file(self, i):
        with h5py.File(self.file_paths[i], "r") as f:
            self.current_file_loaded = i, f["data/data"][:]

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.file_paths) * self.n_batches
        # TODO: Arreglar esto

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        file_index = index // self.n_batches
        current_file_index, _ = self.current_file_loaded
        if file_index != current_file_index:
            self.load_file(file_index)
        _, current_data = self.current_file_loaded
        index_inside_file = index % self.n_batches
        i = index_inside_file * self.batch_size
        data = current_data[i:(i + self.batch_size)]
        data[data > 1.0] = 1.0
        X = data[:, :self.k]
        y = data[:, self.k:self.k*2]
        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            np.random.shuffle(self.file_paths)
            self.current_file_loaded = (None, None)

In [3]:
from tensorflow.keras.losses import binary_crossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

def get_compiled_model():

    inp = layers.Input(shape=(16, 64, 64, 1))


    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(5, 5),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.BatchNormalization()(x)
    """x = layers.ConvLSTM2D(
        filters=64,
        kernel_size=(1, 1),
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)"""
    x = layers.Conv3D(
        filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
    )(x)


    model = Model(inp, x)
    model.compile(
        loss=BinaryCrossentropy(), 
        optimizer=Adam(),
    )
    return model


In [4]:
import os
folder = "../data/exp_pro/GLM-L2-LCFA_8km_5m_boxes_64dt/h5/"
files = np.array([os.path.join(folder, x) for x in os.listdir(folder)][:-1])

train_val_test_split = (.7, .2, .1)

def split(arr, splits):
    arr = np.array(arr)
    idxs = np.arange(len(arr))
    np.random.shuffle(idxs)
    n_splits = list(int(len(arr)*x) for x in splits)
    datasets = []
    start = 0
    n_splits[-1] += len(arr) - sum(n_splits)
    n_splits = tuple(n_splits)
    for split in n_splits:
        datasets.append(arr[idxs[start:start+split]])
        start += split
    return tuple(datasets)

train_files, val_files, test_files = split(files, train_val_test_split)

train_generator = DataGenerator(train_files, k=16)
val_generator = DataGenerator(val_files, k=16)
test_generator = DataGenerator(test_files, k=16)

In [5]:
model = get_compiled_model()

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TerminateOnNaN


filepath = "saved-model-{epoch:02d}-{val_loss:.6f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='auto', save_weights_only=False)
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", patience=5)
terminate_nan = TerminateOnNaN()
model.summary()

In [6]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  []


In [ ]:

epochs = 5

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[
        early_stopping,
        reduce_lr,
        checkpoint,
        terminate_nan,
    ],
)

In [ ]:

model.evaluate(test_generator)